## **GAN を用いた SMILES 文字列生成のコードまとめ**

### **1. 概要**
このコードは、**GAN（Generative Adversarial Network）を用いて SMILES 文字列（化学構造表記）を生成** するものです。  
RDKit を利用し、分子構造の妥当性を考慮しながら SMILES を生成します。  
判別器 (Discriminator) `D` と 生成器 (Generator) `G` の対抗学習により、より本物らしい SMILES を生成することを目指します。

---

### **2. コードの構成**
### **① データの準備**
```python
from google.colab import drive
import pandas as pd
import os

drive.mount('/content/drive')
directory = '/content/drive/My Drive/day6'
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))
```
- Google Drive からデータ（`property_df.csv`）を読み込み。
- `property_df` には、**SMILES 文字列**が含まれる。

---

### **② SMILES データの前処理**
```python
SMILES_COL = "Open Babel SMILES"
SMILES_MAXLEN = 128
LEARNING_RATE = 1e-6
NUM_EPOCHS = 200
BATCH_SIZE = 256
N_HIDDEN = 100
N_LATENT = 100

vocab_freq =  {}
word_length_dist = []
for smile in property_df[SMILES_COL]:
    for s in smile:
        if s not in vocab_freq.keys():
            vocab_freq[s] = 0
        vocab_freq[s] += 1
    word_length_dist.append(len(smile))

vocab = list(vocab_freq.keys())
N_INPUT = len(vocab) * SMILES_MAXLEN
```
- **SMILES の最大長 (`SMILES_MAXLEN = 128`) を設定**。
- **SMILES 文字の vocab（語彙）を作成し、N_INPUT を計算**。

---

### **③ GAN の構築**
```python
G = torch.nn.Sequential(
    torch.nn.Linear(N_LATENT, N_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(N_HIDDEN, N_INPUT),
    torch.nn.Tanh()
).to(device)

D = torch.nn.Sequential(
    torch.nn.Linear(N_INPUT, N_HIDDEN),
    torch.nn.LeakyReLU(0.2),
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),
    torch.nn.LeakyReLU(0.2),
    torch.nn.Linear(N_HIDDEN, 1),
    torch.nn.Sigmoid()
).to(device)
```
- **生成器 (G)**
  - ランダムノイズ `z (N_LATENT)` を入力し、SMILES 文字の one-hot 表現を出力。
- **判別器 (D)**
  - 入力された SMILES が本物かどうかを判定（出力は 0～1）。

---

### **④ One-hot エンコーディングとデータローダーの作成**
```python
def smile2vec(vocab, vecsize, smile):
    vec = []
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]
        if i < len(smile):
            v[vocab.index(smile[i])] = 1
        vec += v
    return vec

X = np.array([smile2vec(vocab, SMILES_MAXLEN, smile) for smile in list(property_df[SMILES_COL])])
X_tensor = torch.from_numpy(X).float()
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
```
- **SMILES を One-hot ベクトルに変換** (`smile2vec`)
- **PyTorch の `DataLoader` を作成**。

---

### **⑤ RDKit を用いた SMILES の修正**
```python
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_best_smile(out_tensor):
    best_smile = ""
    for vec in out_tensor:
        vec = vec.reshape(SMILES_MAXLEN, len(vocab))
        smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
        mol = Chem.MolFromSmiles(smile)
        while not mol:
            if len(smile) == 0: break
            smile = smile[:-1]
            mol = Chem.MolFromSmiles(smile)

        if len(best_smile) < len(smile):
            best_smile = smile

    return best_smile
```
- **生成されたベクトルを SMILES に変換**。
- **RDKit を使って無効な SMILES を修正**。

---

### **⑥ GAN の学習**
```python
for epoch in range(NUM_EPOCHS):
    for i, data in enumerate(data_loader):

        # 判別器の学習
        outputs = D(data[0].to(device))
        real_labels = torch.ones(outputs.shape[0], 1).to(device)
        d_loss_real = criterion(outputs, real_labels)

        z = torch.randn(BATCH_SIZE, N_LATENT).to(device)
        fake_data = G(z)
        outputs = D(fake_data)
        fake_labels = torch.zeros(outputs.shape[0], 1).to(device)
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        if (i % 2) == 0:
            d_loss.backward()
        d_optimizer.step()

        # 生成器の学習
        z = torch.randn(BATCH_SIZE, N_LATENT).to(device)
        fake_data = G(z)
        outputs = D(fake_data)

        real_labels = torch.ones(outputs.shape[0], 1).to(device)
        g_loss = criterion(outputs, real_labels)

        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # ログの出力
        if (i+1) % 10 == 0 and (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.4f}, D(G(z)): {:.4f}'
                  .format(epoch, NUM_EPOCHS, i+1, total_step, d_loss.item(), g_loss.item(),
                          real_score.mean().item(), fake_score.mean().item()))

    if (epoch+1) % 10 == 0:
        print("Epoch[{}/{}], Generated SMILES: {}".format(
            epoch+1, NUM_EPOCHS, get_best_smile(fake_data)))
```
- **判別器 `D` の学習**
  - 本物データを 1 に、偽物データを 0 に判定できるように学習。
- **生成器 `G` の学習**
  - 偽物データを 1 に判定させるように学習。
- **RDKit を用いて生成 SMILES をチェック**。

---

### **7. まとめ**
- **SMILES 文字列を One-hot ベクトル化** し、GAN で学習。
- **判別器 `D` は本物・偽物を判別** し、**生成器 `G` は偽物をより本物らしく改善**。
- **RDKit を利用し、無効な SMILES を修正**。
- **学習が進むと、本物らしい SMILES が生成される**。

このコードを改善することで、より高品質な化学分子を生成できる可能性があります。

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

このコードは、PyTorchを使用して計算デバイス（CPUまたはGPU）を自動的に選択する処理を行っています。

---

### **コードの詳細解説**

```python
import torch
```
- PyTorchのライブラリをインポートします。

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```
- `torch.cuda.is_available()` は、CUDA対応のGPUが使用可能かどうかを確認する関数です。
  - **True** の場合：GPU（CUDA）を利用できるので、`'cuda'` を選択
  - **False** の場合：GPUを利用できないので、`'cpu'` を選択
- `torch.device(...)` は、PyTorchのテンソルをどのデバイス（CPU/GPU）に配置するかを指定するオブジェクトを作成します。

```python
device
```
- `device` の値を表示します。
- 例えば、CUDA対応GPUが使える場合は `device('cuda')`、使えない場合は `device('cpu')` になります。

---

### **動作例**
1. GPUが利用可能な場合:
   ```python
   torch.cuda.is_available()  # → True
   print(device)  # → cuda
   ```
2. GPUが利用不可の場合:
   ```python
   torch.cuda.is_available()  # → False
   print(device)  # → cpu
   ```

---

### **実際の利用例**
この `device` を使って、テンソルやモデルを適切なデバイスに配置できます。

```python
tensor = torch.tensor([1, 2, 3], device=device)  # 指定したデバイスにテンソルを配置
model = MyModel().to(device)  # PyTorchモデルを指定デバイスに転送
```

このコードにより、GPUが使える環境では自動的にGPUを使い、使えない場合はCPUで計算が行われるようになります。

In [ ]:
from google.colab import drive
import pandas as pd
import os

drive.mount('/content/drive')
directory = '/content/drive/My Drive/day6'
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))

Mounted at /content/drive


このコードは、Google Colabを使用してGoogle Driveに保存されたCSVファイルを読み込む処理を行っています。

---

### **コードの詳細解説**

```python
from google.colab import drive
import pandas as pd
import os
```
- `google.colab` の `drive` モジュールをインポートし、Google DriveをColabにマウントするための準備をします。
- `pandas` をインポートしてデータ解析を行えるようにします。
- `os` をインポートして、ファイルパスの操作を簡単にします。

---

```python
drive.mount('/content/drive')
```
- Google Driveを `/content/drive` にマウントします。
- 実行すると、認証のためのリンクが表示され、Googleアカウントで許可を与えると、Google Drive内のファイルにアクセスできるようになります。

---

```python
directory = '/content/drive/My Drive/day6'
```
- Google Drive内のフォルダ `/My Drive/day6` を指定し、このディレクトリ内のファイルを扱う準備をします。

---

```python
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))
```
- `os.path.join(directory, 'property_df.csv')` により、フルパス `/content/drive/My Drive/day6/property_df.csv` を作成します。
- `pd.read_csv()` を使って、指定したCSVファイルをDataFrame (`property_df`) として読み込みます。

---

### **動作の流れ**
1. Google DriveをColabにマウント。
2. Google Driveの特定フォルダ (`day6`) を指定。
3. そのフォルダ内の `property_df.csv` を読み込み、データフレームに格納。

---

### **注意点**
- マウント後にColabのセッションがリセットされると、再マウントが必要になります。
- `property_df.csv` のパスが正しいか確認し、フォルダ `day6` の中に正しく配置されているかチェックしてください。
- `My Drive` の表記は環境によって異なる可能性があるため、`drive.mount('/content/drive')` 後に `!ls /content/drive/My\ Drive` で確認するとよいです。

このコードは、Google Drive上のデータをColabで分析する際に役立ちます！

In [ ]:
SMILES_COL = "Open Babel SMILES"
SMILES_MAXLEN = 128
LEARNING_RATE = 1e-6
NUM_EPOCHS = 200
BATCH_SIZE = 256
N_HIDDEN = 100
N_LATENT = 100

vocab_freq =  {}
word_length_dist = []
for smile in property_df[SMILES_COL]:
    for s in smile:
        if s not in vocab_freq.keys():
            vocab_freq[s] = 0
        vocab_freq[s] += 1
    word_length_dist.append(len(smile))

vocab = list(vocab_freq.keys())
N_INPUT = len(vocab)*SMILES_MAXLEN


このコードは、SMILES（Simplified Molecular Input Line Entry System）表記を用いた化学データを扱うための前処理を行い、ニューラルネットワークのハイパーパラメータを設定する処理を含んでいます。  

---

## **コードの詳細解説**

### **1. 定数の設定**
```python
SMILES_COL = "Open Babel SMILES"
SMILES_MAXLEN = 128
LEARNING_RATE = 1e-6
NUM_EPOCHS = 200
BATCH_SIZE = 256
N_HIDDEN = 100
N_LATENT = 100
```
- `SMILES_COL`：DataFrame (`property_df`) 内でSMILES表記の分子構造が格納されているカラム名。
- `SMILES_MAXLEN`：SMILES文字列の最大長。最大128文字までの分子表記を考慮。
- `LEARNING_RATE`：学習率（`1e-6 = 0.000001`）、ニューラルネットワークの学習のステップサイズ。
- `NUM_EPOCHS`：学習を繰り返す回数（エポック数）。
- `BATCH_SIZE`：ミニバッチ学習時のデータサイズ（256個ずつ学習）。
- `N_HIDDEN`：隠れ層のニューロン数（100）。
- `N_LATENT`：潜在変数（latent vector）の次元数（100）。

---

### **2. SMILESデータの前処理**
```python
vocab_freq =  {}
word_length_dist = []
```
- `vocab_freq`：SMILES文字列に含まれる各文字の出現回数を記録する辞書。
- `word_length_dist`：各SMILES文字列の長さを格納するリスト。

```python
for smile in property_df[SMILES_COL]:
    for s in smile:
        if s not in vocab_freq.keys():
            vocab_freq[s] = 0
        vocab_freq[s] += 1
    word_length_dist.append(len(smile))
```
このループでは、以下の処理を行います：
1. `property_df[SMILES_COL]` から各SMILES文字列を取得。
2. 文字列内の各文字 (`s`) を確認し、`vocab_freq` に登録。
   - もし辞書に存在しなければ初期値 `0` を設定。
   - 出現回数をカウント (`+= 1`)。
3. 各SMILESの長さを `word_length_dist` に保存。

この処理が完了すると：
- `vocab_freq` は、SMILESに含まれる文字（原子や結合の表現）ごとの出現頻度を保持。
- `word_length_dist` には、すべてのSMILESの長さがリストとして格納される。

---

### **3. 語彙（vocab）の作成**
```python
vocab = list(vocab_freq.keys())
```
- `vocab` は、SMILES文字列に含まれる一意な文字のリスト。
- 例えば、SMILESに `C`, `O`, `N`, `=`, `#`, `[`, `]` などの文字が含まれていた場合、`vocab` は `['C', 'O', 'N', '=', '#', '[', ']']` というリストになる。

```python
N_INPUT = len(vocab) * SMILES_MAXLEN
```
- `N_INPUT` は、ニューラルネットワークの入力次元数を計算。
  - `len(vocab)` は、SMILESに登場するユニークな文字の数（語彙サイズ）。
  - `SMILES_MAXLEN` は、SMILESの最大長。
  - 例えば、`len(vocab) = 50` で `SMILES_MAXLEN = 128` の場合、`N_INPUT = 50 × 128 = 6400` となる。

---

## **まとめ**
- **定数の設定**：ニューラルネットワークのハイパーパラメータを定義。
- **SMILESデータの処理**：
  - 各SMILES文字列のユニークな文字（語彙）を取得。
  - 各文字の出現回数をカウント。
  - 各SMILESの長さを記録。
- **入力次元の計算**：
  - 語彙サイズとSMILESの最大長を用いて、ニューラルネットワークの入力サイズ (`N_INPUT`) を決定。

この前処理は、SMILESデータを機械学習モデルに入力できる形（例えば、one-hotベクトルや埋め込みベクトル）に変換するための準備として重要です。

In [ ]:
G = torch.nn.Sequential(
    torch.nn.Linear(N_LATENT, N_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(N_HIDDEN, N_INPUT),
    torch.nn.Tanh()).to(device)

D = torch.nn.Sequential(
    torch.nn.Linear(N_INPUT, N_HIDDEN),
    torch.nn.LeakyReLU(0.2),
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),
    torch.nn.LeakyReLU(0.2),
    torch.nn.Linear(N_HIDDEN, 1),
    torch.nn.Sigmoid()).to(device)

criterion = torch.nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=LEARNING_RATE)
g_optimizer = torch.optim.Adam(G.parameters(), lr=LEARNING_RATE)

このコードは、**Generative Adversarial Network (GAN)** のジェネレーター (G) とディスクリミネーター (D) を定義し、それらを学習するための損失関数と最適化手法を設定しています。

---

## **1. ジェネレーター (G) の定義**
```python
G = torch.nn.Sequential(
    torch.nn.Linear(N_LATENT, N_HIDDEN),  # 入力層（潜在ベクトル → 隠れ層）
    torch.nn.ReLU(),                      # 活性化関数（ReLU）
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),  # 隠れ層（中間層）
    torch.nn.ReLU(),                      # 活性化関数（ReLU）
    torch.nn.Linear(N_HIDDEN, N_INPUT),   # 出力層（隠れ層 → 出力層）
    torch.nn.Tanh()                       # 活性化関数（Tanh）
).to(device)
```
### **ジェネレーターの構造**
- `N_LATENT`（潜在ベクトル）を入力し、`N_INPUT`（SMILESデータの1次元表現）を出力。
- **3層の線形変換 (Linear) を持つネットワーク**
  1. **(N_LATENT → N_HIDDEN)**
  2. **(N_HIDDEN → N_HIDDEN)**
  3. **(N_HIDDEN → N_INPUT)**
- 活性化関数：
  - **ReLU**：隠れ層で使用（勾配消失問題を防ぐため）
  - **Tanh**：出力層で使用（GANでは出力を -1～1 の範囲にするため一般的）

### **動作イメージ**
- `G` はランダムなノイズ (`N_LATENT` 次元) を受け取り、 `N_INPUT` 次元のデータ（生成したSMILESの数値表現）を出力する。
- **本物っぽいデータを生成する役割** を持つ。

---

## **2. ディスクリミネーター (D) の定義**
```python
D = torch.nn.Sequential(
    torch.nn.Linear(N_INPUT, N_HIDDEN),   # 入力層（SMILESデータ → 隠れ層）
    torch.nn.LeakyReLU(0.2),              # 活性化関数（LeakyReLU）
    torch.nn.Linear(N_HIDDEN, N_HIDDEN),  # 隠れ層
    torch.nn.LeakyReLU(0.2),              # 活性化関数（LeakyReLU）
    torch.nn.Linear(N_HIDDEN, 1),         # 出力層（1つのスカラ値: 本物 or 偽）
    torch.nn.Sigmoid()                    # 活性化関数（Sigmoidで確率を出力）
).to(device)
```
### **ディスクリミネーターの構造**
- **3層の線形変換 (Linear) を持つネットワーク**
  1. **(N_INPUT → N_HIDDEN)**
  2. **(N_HIDDEN → N_HIDDEN)**
  3. **(N_HIDDEN → 1)**
- 活性化関数：
  - **LeakyReLU(0.2)**：負の値も少し通すことで学習の停滞を防ぐ（`α=0.2`）。
  - **Sigmoid**：最後の出力を `[0,1]` の確率に変換（`1 = 本物, 0 = 偽`）。

### **動作イメージ**
- `D` は `N_INPUT`（SMILESデータの数値表現）を受け取り、それが本物 (`1`) か偽物 (`0`) かを確率で出力。
- **本物と偽物を見分ける役割** を持つ。

---

## **3. 損失関数の定義**
```python
criterion = torch.nn.BCELoss()
```
- **BCE (Binary Cross Entropy) 損失関数**：
  - ディスクリミネーター `D` の出力（確率）と、ラベル（本物: `1`, 偽物: `0`）の誤差を測る。
  - GANで一般的に使用される損失関数。

---

## **4. 最適化手法の定義**
```python
d_optimizer = torch.optim.Adam(D.parameters(), lr=LEARNING_RATE)
g_optimizer = torch.optim.Adam(G.parameters(), lr=LEARNING_RATE)
```
- `torch.optim.Adam`：適応学習率を持つ Adam 最適化アルゴリズムを使用。
- `LEARNING_RATE`（学習率）を指定。
- `D.parameters()`：ディスクリミネーターのパラメータを更新。
- `G.parameters()`：ジェネレーターのパラメータを更新。

---

## **GAN の学習の流れ**
1. **ディスクリミネーターの学習**
   - `G` が生成した偽データと、本物のデータを `D` に入力。
   - `D` が正しく分類できるように学習。
   - **本物には 1、偽物には 0 を出力するように損失関数 BCE で学習**。

2. **ジェネレーターの学習**
   - `G` の出力（偽データ）を `D` に入力し、`D(G(z))` の出力を `1` に近づけるように学習。
   - **偽データを本物に近づけるように `G` を改善**。

---

## **まとめ**
- **`G` (ジェネレーター)**：
  - ノイズ (`N_LATENT`) を入力し、本物っぽい `N_INPUT` のデータを生成するネットワーク。
  - ReLU（隠れ層）＋Tanh（出力層）を使用。

- **`D` (ディスクリミネーター)**：
  - 入力 (`N_INPUT`) を受け取り、それが本物か偽物かを判別するネットワーク。
  - LeakyReLU（隠れ層）＋Sigmoid（出力層）を使用。

- **損失関数**：
  - `BCE (Binary Cross Entropy) Loss` を使用。

- **最適化手法**：
  - `Adam` を使用し、ジェネレーターとディスクリミネーターをそれぞれ更新。

このコードは、**SMILES表記の分子データを生成するための GAN** のベースモデルを作成するものです！

In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

def smile2vec(vocab, vecsize, smile):
    vec = []
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]
        if i < len(smile):
            v[vocab.index(smile[i])] = 1
        vec += v
    return vec


X = np.array([smile2vec(vocab, SMILES_MAXLEN, smile) for smile in list(property_df[SMILES_COL])])
X_tensor = torch.from_numpy(X).float()
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


このコードは、**SMILES表記の化学分子データを数値ベクトルに変換し、それをPyTorchのデータローダー（DataLoader）で扱える形にする処理** を行っています。  

---

## **1. 必要なライブラリのインポート**
```python
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
```
- `numpy`：データの数値計算（行列操作）に使用。
- `torch`：PyTorchのテンソル操作に使用。
- `torch.utils.data.TensorDataset`：PyTorchのデータセットクラス。
- `torch.utils.data.DataLoader`：データセットをバッチ単位で扱えるようにするためのデータローダー。

---

## **2. SMILES文字列をベクトル化する関数**
```python
def smile2vec(vocab, vecsize, smile):
    vec = []  # ベクトルを格納するリスト
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]  # 語彙サイズ分のゼロベクトルを作成
        if i < len(smile):  # SMILES文字列の範囲内であれば
            v[vocab.index(smile[i])] = 1  # one-hotエンコーディング
        vec += v  # ベクトルを連結
    return vec
```
### **関数の詳細**
- `vocab`: SMILES表記に含まれるユニークな文字のリスト（語彙）。
- `vecsize`: 最大SMILES長 (`SMILES_MAXLEN`)。
- `smile`: 変換するSMILES文字列。

**処理の流れ**
1. `vecsize`（= `SMILES_MAXLEN`）分のループを回す。
2. `vocab` のサイズだけ `0` のリスト（one-hotベクトル）を作る。
3. `smile` の `i` 番目の文字が `vocab` にある場合、その位置を `1` にする（one-hotベクトル）。
4. すべてのone-hotベクトルを連結して1つの長いベクトルにする。

### **例**
#### **入力**
```python
vocab = ['C', 'O', 'N']
SMILES_MAXLEN = 5
smile = "CON"
```
#### **処理**
```
C → [1, 0, 0], O → [0, 1, 0], N → [0, 0, 1]
```
#### **出力**
```
[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]  # (5 * 3 = 15次元)
```
このように、固定長 (`SMILES_MAXLEN`) のベクトルを作成する。

---

## **3. SMILESデータのベクトル化**
```python
X = np.array([smile2vec(vocab, SMILES_MAXLEN, smile) for smile in list(property_df[SMILES_COL])])
```
- `property_df[SMILES_COL]` の全てのSMILES文字列を `smile2vec()` を使って変換。
- `np.array()` を使って、リストをNumPy配列に変換。

---

## **4. PyTorchテンソルに変換**
```python
X_tensor = torch.from_numpy(X).float()
```
- `numpy` の `X` を **PyTorchのテンソル** に変換。
- `.float()` にすることで、ニューラルネットワークで処理しやすくする。

---

## **5. データセットとデータローダーの作成**
```python
dataset = TensorDataset(X_tensor)
```
- `X_tensor` を `TensorDataset` に変換し、PyTorchのデータセットとして扱えるようにする。

```python
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
```
- `DataLoader` を作成し、データをバッチ単位で処理できるようにする。
- `batch_size=BATCH_SIZE`（例: 256）: 1回の学習で使うデータ数。
- `shuffle=True`：データをランダムに並び替える。

---

## **まとめ**
1. **SMILESをone-hotベクトル化**
   - `smile2vec()` で `SMILES_MAXLEN × len(vocab)` のベクトルを作成。
2. **データの変換**
   - `NumPy配列` → `PyTorchテンソル` に変換。
3. **データセット & データローダーの作成**
   - `TensorDataset` を作成し、`DataLoader` でミニバッチ処理を可能にする。

このコードは、GAN（または他の機械学習モデル）の学習に適した形式にSMILESデータを変換するための前処理を行っています！

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 34.8 MB/s eta 0:00:00


このコードは、Google Colab などの環境で **RDKit** という化学情報処理ライブラリをインストールするためのコマンドです。

---

## **1. コードの詳細**
```python
!pip install rdkit
```
- `!`（エクスクラメーションマーク）：Pythonスクリプト内でシェルコマンドを実行するための記号。
- `pip install rdkit`：`rdkit` ライブラリを `pip`（Pythonのパッケージ管理システム）を使ってインストール。

---

## **2. RDKitとは？**
**RDKit（The RDKit: Open-Source Cheminformatics）** は、分子データを扱うための強力な **化学情報処理ライブラリ** です。  
SMILES表記の分子を分子構造に変換したり、分子の特徴量を計算したりするのに使われます。

### **主な機能**
- **SMILES表記** の分子データの処理
- **分子構造の可視化**
- **物性予測**（分子量、LogP など）
- **分子フィンガープリントの生成**（類似分子検索）
- **ケモインフォマティクス**（分子データ解析）

---

## **3. インストール後の確認**
インストールが成功したか確認するには、以下のコードを実行します。
```python
import rdkit
from rdkit import Chem

print(rdkit.__version__)  # RDKitのバージョンを表示
```
問題なくバージョンが表示されれば、正常にインストールされています。

---

## **4. 注意点**
- **Google Colab** では、`rdkit` はデフォルトではインストールされていないため、このコマンドが必要です。
- **ローカル環境** では、Anaconda を使う場合は `conda install -c conda-forge rdkit` を推奨。

---

### **まとめ**
- `!pip install rdkit` は、**RDKitライブラリをPython環境にインストール** するコマンド。
- **RDKit** は、SMILES表記の分子データを処理・分析するための **化学情報処理ライブラリ**。
- Google Colab では事前にインストールが必要。

In [ ]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_best_smile(out_tensor):
    best_smile = ""
    for vec in out_tensor:
        vec = vec.reshape(SMILES_MAXLEN, len(vocab))
        smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
        mol = Chem.MolFromSmiles(smile)
        while not mol:
            if len(smile) == 0: break
            smile = smile[:-1]
            mol = Chem.MolFromSmiles(smile)

        if len(best_smile) < len(smile):
            best_smile = smile

    return best_smile

このコードは、**GAN などのモデルの出力テンソルを SMILES 文字列に変換し、最も良い（適切な）SMILES を選択する処理** を行っています。

---

## **1. 必要なライブラリのインポート**
```python
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
```
- `rdkit.Chem`:
  - **RDKit の化学情報処理モジュール**。
  - `Chem.MolFromSmiles(smile)` を使って、SMILES 文字列を分子オブジェクトに変換する。
- `RDLogger.DisableLog('rdApp.*')`:
  - RDKit のログ出力を抑制（警告メッセージを非表示にする）。

---

## **2. `get_best_smile(out_tensor)` 関数の解説**
```python
def get_best_smile(out_tensor):
    best_smile = ""  # 最も良いSMILESを保存する変数
```
- `out_tensor`: GAN などのモデルが出力したテンソル（複数の SMILES の候補が含まれる）。
- `best_smile`: 最も良い SMILES 文字列を保存するための変数。

---

## **3. `out_tensor` の各データを処理**
```python
for vec in out_tensor:
    vec = vec.reshape(SMILES_MAXLEN, len(vocab))  # 形状を変換 (SMILES_MAXLEN, vocab_size)
    smile = "".join([vocab[torch.argmax(v).item()] for v in vec])  # One-hot を SMILES に変換
```
### **処理内容**
1. `vec.reshape(SMILES_MAXLEN, len(vocab))`
   - `out_tensor` から **1つのSMILESベクトルを取得し、2次元の形状 (SMILES_MAXLEN × vocab_size) に変換**。
   - `vec.shape = (SMILES_MAXLEN × vocab_size)` のように 1次元になっている可能性があるので、元の形に戻す。

2. **One-hot ベクトルを SMILES 文字列に変換**
   ```python
   smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
   ```
   - `torch.argmax(v).item()` で **各文字の最大値のインデックス** を取得（one-hot エンコーディングから復元）。
   - `vocab[index]` を取得し、リスト内包表記で SMILES 文字列を生成。
   - `"".join([...])` で文字列として結合。

---

## **4. SMILES の修正**
```python
mol = Chem.MolFromSmiles(smile)  # RDKit を使って分子オブジェクトを作成
while not mol:  # もしSMILESが無効なら修正する
    if len(smile) == 0: break  # 空になったら終了
    smile = smile[:-1]  # 末尾の1文字を削除
    mol = Chem.MolFromSmiles(smile)  # 再度チェック
```
### **処理内容**
- `Chem.MolFromSmiles(smile)` を使い、RDKit で **SMILES 文字列を分子オブジェクト (`mol`) に変換**。
- もし `mol` が `None`（無効なSMILES）なら、**末尾を1文字ずつ削除** して有効なSMILESに修正。

---

## **5. 最も良いSMILESを選択**
```python
if len(best_smile) < len(smile):  # より長い有効なSMILESを採用
    best_smile = smile
```
- **最も長い有効な SMILES を選択**。
  - 途中で切られた短いSMILESではなく、できるだけ長い文字列を保持。

---

## **6. 最終的なSMILESを返す**
```python
return best_smile
```
- `best_smile`（最も適切なSMILES）を返す。

---

## **関数の動作例**
### **入力**
```python
out_tensor = torch.tensor([
    [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...],  # one-hotベクトル (flattened)
])
```
### **処理**
1. **One-hot ベクトル → 文字列**
   ```
   "CCO(無効)O" → 修正 → "CCO"
   ```
2. **最も長い有効なSMILESを選択**
   - `best_smile = "CCO"`

### **出力**
```python
"CCO"
```
---

## **まとめ**
- **GANの出力テンソルをSMILES文字列に変換** する関数。
- **無効なSMILESを修正** し、最も長い有効なSMILESを選択。
- **RDKit (`Chem.MolFromSmiles`) を使って分子として有効か確認** し、不適切なSMILESを修正。

In [ ]:
d_losses = []
g_losses = []
real_scores = []
fake_scores = []

total_step = len(data_loader)
for epoch in range(NUM_EPOCHS):
    for i, data in enumerate(data_loader):

        # 判別器の学習
        outputs = D(data[0].to(device))
        real_labels = torch.ones(outputs.shape[0], 1).to(device)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        z = torch.randn(BATCH_SIZE, N_LATENT).to(device)
        fake_data = G(z)
        outputs = D(fake_data)
        fake_labels = torch.zeros(outputs.shape[0], 1).to(device)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        if (i%2) == 0:
            d_loss.backward()
        d_optimizer.step()

        # 生成器の学習
        z = torch.randn(BATCH_SIZE, N_LATENT).to(device)
        fake_data = G(z)
        outputs = D(fake_data)

        real_labels = torch.ones(outputs.shape[0], 1).to(device)
        g_loss = criterion(outputs, real_labels)

        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # 結果の記録
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())
        real_scores.append(real_score.mean().item())
        fake_scores.append(fake_score.mean().item())
        if (i+1) % 10 == 0 and (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.4f}, D(G(z)): {:.4f}'
                  .format(epoch, NUM_EPOCHS, i+1, total_step, d_loss.item(), g_loss.item(),
                          real_score.mean().item(), fake_score.mean().item()))

    if (epoch+1) % 10 == 0:
        print("Epoch[{}/{}], Generated SMILES: {}".format(
            epoch+1, NUM_EPOCHS, get_best_smile(fake_data)))

Epoch [9/200], Step [10/35], d_loss: 1.3717, g_loss: 0.6960, D(x): 0.5060, D(G(z)): 0.4987
Epoch [9/200], Step [20/35], d_loss: 1.3713, g_loss: 0.6966, D(x): 0.5060, D(G(z)): 0.4985
Epoch [9/200], Step [30/35], d_loss: 1.3707, g_loss: 0.6966, D(x): 0.5061, D(G(z)): 0.4983
Epoch[10/200], Generated SMILES: OC
Epoch [19/200], Step [10/35], d_loss: 1.3535, g_loss: 0.7125, D(x): 0.5070, D(G(z)): 0.4905
Epoch [19/200], Step [20/35], d_loss: 1.3521, g_loss: 0.7128, D(x): 0.5073, D(G(z)): 0.4900
Epoch [19/200], Step [30/35], d_loss: 1.3518, g_loss: 0.7135, D(x): 0.5072, D(G(z)): 0.4898
Epoch[20/200], Generated SMILES: C\S
Epoch [29/200], Step [10/35], d_loss: 1.3334, g_loss: 0.7305, D(x): 0.5084, D(G(z)): 0.4816
Epoch [29/200], Step [20/35], d_loss: 1.3332, g_loss: 0.7314, D(x): 0.5084, D(G(z)): 0.4814
Epoch [29/200], Step [30/35], d_loss: 1.3327, g_loss: 0.7317, D(x): 0.5085, D(G(z)): 0.4812
Epoch[30/200], Generated SMILES: O
Epoch [39/200], Step [10/35], d_loss: 1.3149, g_loss: 0.7468, D(x):

このコードは、**GAN（Generative Adversarial Network）を使って SMILES 文字列を生成するための学習ループ** です。  
特に、**判別器 (Discriminator, D) と 生成器 (Generator, G) の学習** を行っています。

---

## **1. 変数の初期化**
```python
d_losses = []      # 判別器の損失を記録
g_losses = []      # 生成器の損失を記録
real_scores = []   # 本物データの判別結果
fake_scores = []   # 偽物データの判別結果

total_step = len(data_loader)  # 1エポックのステップ数（バッチ数）
```
- `d_losses`、`g_losses` は、それぞれ判別器・生成器の損失値を記録するリスト。
- `real_scores`、`fake_scores` は、本物と偽物データの判別スコア（0～1）を記録。
- `total_step` は、1エポックの総バッチ数（`data_loader` のバッチ数）。

---

## **2. 学習ループ**
```python
for epoch in range(NUM_EPOCHS):
    for i, data in enumerate(data_loader):
```
- `NUM_EPOCHS` 回のエポック（学習回数）を繰り返す。
- `data_loader` からデータを取得し、`i` はバッチのインデックス。

---

## **3. 判別器 (D) の学習**
```python
# 本物データを D に入力
outputs = D(data[0].to(device))  
real_labels = torch.ones(outputs.shape[0], 1).to(device)  # 本物データのラベル (1)
d_loss_real = criterion(outputs, real_labels)  # 本物データの損失
real_score = outputs  # 本物データの判別結果
```
- **本物データを D に入力し、ラベル 1 との損失を計算**。

```python
# 偽物データを生成し D に入力
z = torch.randn(BATCH_SIZE, N_LATENT).to(device)  # ランダムノイズを生成
fake_data = G(z)  # 生成器 G で偽物データを生成
outputs = D(fake_data)  # 偽物データを D に入力
fake_labels = torch.zeros(outputs.shape[0], 1).to(device)  # 偽物データのラベル (0)
d_loss_fake = criterion(outputs, fake_labels)  # 偽物データの損失
fake_score = outputs  # 偽物データの判別結果
```
- **ノイズ `z` から生成器 `G` を通じて偽物データを生成**。
- **D に入力し、偽物データが 0 に近いスコアになるよう学習**。

```python
d_loss = d_loss_real + d_loss_fake  # 本物・偽物の合計損失
d_optimizer.zero_grad()
g_optimizer.zero_grad()
if (i % 2) == 0:  # 2回に1回だけ更新
    d_loss.backward()
d_optimizer.step()
```
- 判別器の損失を計算し、**2回に1回だけ D を更新**（判別器の学習が強すぎるのを防ぐ）。

---

## **4. 生成器 (G) の学習**
```python
z = torch.randn(BATCH_SIZE, N_LATENT).to(device)
fake_data = G(z)
outputs = D(fake_data)

real_labels = torch.ones(outputs.shape[0], 1).to(device)  # 偽物データを「本物」と判定させる
g_loss = criterion(outputs, real_labels)  # 生成器の損失 (D を騙せるか)
```
- 新たにノイズ `z` を生成し、`G` で偽物データを作る。
- 偽物データを `D` に入力し、**本物 (1) だと判定させるように学習**。

```python
d_optimizer.zero_grad()
g_optimizer.zero_grad()
g_loss.backward()
g_optimizer.step()
```
- **生成器 `G` のパラメータを更新**。

---

## **5. 損失・スコアの記録**
```python
d_losses.append(d_loss.item())
g_losses.append(g_loss.item())
real_scores.append(real_score.mean().item())
fake_scores.append(fake_score.mean().item())
```
- 判別器・生成器の損失、D の出力スコア（本物・偽物）をリストに保存。

---

## **6. ログの出力**
```python
if (i+1) % 10 == 0 and (epoch+1) % 10 == 0:
    print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.4f}, D(G(z)): {:.4f}'
          .format(epoch, NUM_EPOCHS, i+1, total_step, d_loss.item(), g_loss.item(),
                  real_score.mean().item(), fake_score.mean().item()))
```
- **10ステップごと & 10エポックごとに学習状況を表示**。
- `D(x)`: 本物データの平均スコア（1 に近いほど良い）。
- `D(G(z))`: 偽物データの平均スコア（0 に近いほど良い）。

---

## **7. SMILES 文字列の生成**
```python
if (epoch+1) % 10 == 0:
    print("Epoch[{}/{}], Generated SMILES: {}".format(
        epoch+1, NUM_EPOCHS, get_best_smile(fake_data)))
```
- **10エポックごとに、生成した SMILES を `get_best_smile` で変換して表示**。

---

## **まとめ**
- **判別器 (D) の学習**
  - 本物データ (`data[0]`) を入力し、1 に近づける。
  - 偽物データ (`G(z)`) を入力し、0 に近づける。
  - `D` を 2回に1回だけ更新（学習バランス調整）。

- **生成器 (G) の学習**
  - `D(G(z))` の出力を **1 に近づけるように学習**（D を騙せるようにする）。

- **結果の記録**
  - 損失 (`d_loss, g_loss`)、スコア (`D(x), D(G(z))`) を記録。
  - 10エポックごとに **生成した SMILES を出力**。